In [25]:
import numpy as np
import pandas as pd

# FOSSP 实例参数
n, m = 2, 2  # 作业数，机器数
l = 2  # 二进制位数
P = np.array([[1, 2], [2, 1]])  # 加工时间矩阵
K1, K2 = 100, 100  # 增大10倍  # 惩罚系数

# 总变量数
N = n * m + l + m * l  # 17 个自旋

# 初始化 QUBO 矩阵
Q = np.zeros((N, N))

# 变量索引
x_idx = [(i * m + j) for i in range(n) for j in range(m)]  # 0-8
z_idx = [n * m + h for h in range(l)]  # 9-10
tau_idx = [(n * m + l) + j * l + h for j in range(m) for h in range(l)]  # 11-16

# 项 1：最小化 C_max
for h in range(l):
    Q[z_idx[h], z_idx[h]] += 2 ** h

# 项 2：约束 sum_j x_ij = 1
for i in range(n):
    for j in range(m):
        idx_j = x_idx[i * m + j]
        Q[idx_j, idx_j] += -2 * K1  # 线性项
        for k in range(j + 1, m):
            idx_k = x_idx[i * m + k]
            Q[idx_j, idx_k] += 2 * K1
            Q[idx_k, idx_j] += 2 * K1

# 项 3：机器完工时间约束
for j in range(m):
    # A_j^2 = (sum_i x_ij p_ij)^2
    for i in range(n):
        idx_i = x_idx[i * m + j]
        Q[idx_i, idx_i] += K2 * P[i, j]**2  # x_ij^2
        for k in range(i + 1, n):
            idx_k = x_idx[k * m + j]
            Q[idx_i, idx_k] += 2 * K2 * P[i, j] * P[k, j]
            Q[idx_k, idx_i] += 2 * K2 * P[i, j] * P[k, j]
    # B_j^2 = (sum_h 2^h (tau_jh - z_h))^2
    for h in range(l):
        tau_h = tau_idx[j * l + h]
        z_h = z_idx[h]
        Q[tau_h, tau_h] += K2 * (2**h)**2  # tau_jh^2
        Q[z_h, z_h] += K2 * (2**h)**2  # z_h^2
        Q[tau_h, z_h] += -2 * K2 * (2**h)**2  # -2 tau_jh z_h
        Q[z_h, tau_h] += -2 * K2 * (2**h)**2  # 对称项
        for k in range(l):
            if h != k:
                tau_k = tau_idx[j * l + k]
                z_k = z_idx[k]
                Q[tau_h, tau_k] += 2 * K2 * 2**h * 2**k
                Q[tau_k, tau_h] += 2 * K2 * 2**h * 2**k
                Q[z_h, z_k] += 2 * K2 * 2**h * 2**k
                Q[z_k, z_h] += 2 * K2 * 2**h * 2**k
                Q[tau_h, z_k] += -2 * K2 * 2**h * 2**k
                Q[z_k, tau_h] += -2 * K2 * 2**h * 2**k
    # 2 A_j B_j
    for i in range(n):
        idx_i = x_idx[i * m + j]
        for h in range(l):
            tau_h = tau_idx[j * l + h]
            z_h = z_idx[h]
            Q[idx_i, tau_h] += 2 * K2 * P[i, j] * 2**h
            Q[tau_h, idx_i] += 2 * K2 * P[i, j] * 2**h
            Q[idx_i, z_h] += -2 * K2 * P[i, j] * 2**h
            Q[z_h, idx_i] += -2 * K2 * P[i, j] * 2**h

In [45]:
# 量化到 8-bit
Q_max = np.max(np.abs(Q))
scale = 127 / Q_max if Q_max > 0 else 1
Q_quantized = np.round(Q * scale).astype(np.int8)
print("Quantized Q (8-bit):\n", Q_quantized)
print("Scale factor:", scale)

Quantized Q (8-bit):
 [[ -8  16  32   0 -16 -32  16  32   0   0]
 [ 16  16   0  32 -32 -64   0   0  32  64]
 [ 32   0  16  16 -32 -64  32  64   0   0]
 [  0  32  16  -8 -16 -32   0   0  16  32]
 [-16 -32 -32 -16  16 127 -16 -32 -16 -32]
 [-32 -64 -64 -32 127  64 -32 -64 -32 -64]
 [ 16   0  32   0 -16 -32   8  64   0   0]
 [ 32   0  64   0 -32 -64  64  32   0   0]
 [  0  32   0  16 -16 -32   0   0   8  64]
 [  0  64   0  32 -32 -64   0   0  64  32]]
Scale factor: 127.0


In [26]:
Q = Q / np.max(np.abs(Q))

In [47]:
print("J shape:", J.shape, "Max |J|:", np.max(np.abs(J)))
print("h shape:", h.shape, "Max |h|:", np.max(np.abs(h)))

J shape: (10, 10) Max |J|: 0.25
h shape: (10,) Max |h|: 0.25


In [30]:
import numpy as np

def qubo_to_ising(Q):
    """
    Convert a QUBO matrix to an Ising model.
    
    Args:
        Q (np.ndarray): Symmetric QUBO matrix (N x N), where x_i in {0,1}.
    
    Returns:
        J (np.ndarray): Symmetric coupling matrix (N x N) for Ising model, s_i in {-1,1}.
        h (np.ndarray): External field vector (N) for Ising model.
    """
    # Input validation
    if not isinstance(Q, np.ndarray) or Q.ndim != 2 or Q.shape[0] != Q.shape[1]:
        raise ValueError("Q must be a square NumPy array")
    N = Q.shape[0]
    if not np.allclose(Q, Q.T):
        print("Warning: Q is not symmetric, symmetrizing Q")
        Q = (Q + Q.T) / 2

    # Normalize Q to avoid numerical issues
    Q_max = np.max(np.abs(Q))
    if Q_max > 0:
        Q = Q / Q_max

    # Initialize Ising parameters
    J = np.zeros((N, N))
    h = np.zeros(N)

    # Compute external field h
    for i in range(N):
        h[i] = Q[i, i] / 2 + (np.sum(Q[i, :]) - Q[i, i]) / 4

    # Compute coupling matrix J (upper triangle)
    for i in range(N):
        for j in range(i + 1, N):
            J[i, j] = Q[i, j] / 4

    # Symmetrize J
    J = J + J.T

    # If Q was normalized, scale J and h back
    if Q_max > 0:
        J *= Q_max
        h *= Q_max

    return J, h

# Example usage (assuming Q is from previous FOSSP QUBO)
# Q = np.array([...])  # Your QUBO matrix
# J, h = qubo_to_ising(Q)
# print("Ising J matrix:\n", J)
# print("Ising h vector:\n", h)

In [31]:
J, h = qubo_to_ising(Q)
print("J:\n", J)
print("h:\n", h)

J:
 [[ 0.       0.03125  0.0625   0.      -0.03125 -0.0625   0.03125  0.0625
   0.       0.     ]
 [ 0.03125  0.       0.       0.0625  -0.0625  -0.125    0.       0.
   0.0625   0.125  ]
 [ 0.0625   0.       0.       0.03125 -0.0625  -0.125    0.0625   0.125
   0.       0.     ]
 [ 0.       0.0625   0.03125  0.      -0.03125 -0.0625   0.       0.
   0.03125  0.0625 ]
 [-0.03125 -0.0625  -0.0625  -0.03125  0.       0.25    -0.03125 -0.0625
  -0.03125 -0.0625 ]
 [-0.0625  -0.125   -0.125   -0.0625   0.25     0.      -0.0625  -0.125
  -0.0625  -0.125  ]
 [ 0.03125  0.       0.0625   0.      -0.03125 -0.0625   0.       0.125
   0.       0.     ]
 [ 0.0625   0.       0.125    0.      -0.0625  -0.125    0.125    0.
   0.       0.     ]
 [ 0.       0.0625   0.       0.03125 -0.03125 -0.0625   0.       0.
   0.       0.125  ]
 [ 0.       0.125    0.       0.0625  -0.0625  -0.125    0.       0.
   0.125    0.     ]]
h:
 [ 0.0625     0.15625    0.15625    0.0625    -0.0621875 -0.249375
  0.1562

In [27]:
# # 转换为伊辛模型
# J = np.zeros((N, N))
# h = np.zeros(N)
# for i in range(N):
#     h[i] = Q[i, i] / 2 + sum(Q[i, j] / 4 for j in range(N) if j != i)
#     for j in range(i + 1, N):
#         J[i, j] = Q[i, j] / 4
# J = J + J.T

In [49]:
# 验证 QUBO 矩阵
print(f"QUBO 矩阵形状: {Q.shape}")
print(f"QUBO 矩阵是否对称: {np.allclose(Q, Q.T)}")
print(f"QUBO 矩阵非零元素数: {np.count_nonzero(Q)}")
print("QUBO 矩阵前 5 行 5 列:\n", Q[:5, :5])

# 保存 QUBO 矩阵到 CSV
# labels = [f"x_{i + 1}{j + 1}" for i in range(n) for j in range(m)] + \
#          [f"z_{h}" for h in range(l)] + \
#          [f"tau_{j + 1}{h}" for j in range(m) for h in range(l)]
labels = [f"x_{i + 1}{j + 1}" for i in range(n) for j in range(m)] + \
         [f"z_{h}" for h in range(l)] + \
         [f"tau_{j + 1}{h}" for j in range(m) for h in range(l)]
# Q_df = pd.DataFrame(Q, index=labels, columns=labels)
Q_df = pd.DataFrame(Q_quantized)
Q_df.to_csv("qubo_matrix_corrected.csv", index=False, header=False)
# Q_df.to_csv("qubo_matrix_corrected.csv")
print("标准 QUBO 矩阵已保存为 qubo_matrix_corrected.csv")

QUBO 矩阵形状: (10, 10)
QUBO 矩阵是否对称: True
QUBO 矩阵非零元素数: 72
QUBO 矩阵前 5 行 5 列:
 [[-0.0625    0.125     0.25      0.       -0.125   ]
 [ 0.125     0.125     0.        0.25     -0.25    ]
 [ 0.25      0.        0.125     0.125    -0.25    ]
 [ 0.        0.25      0.125    -0.0625   -0.125   ]
 [-0.125    -0.25     -0.25     -0.125     0.125625]]
标准 QUBO 矩阵已保存为 qubo_matrix_corrected.csv


In [41]:
from cim_optimizer.solve_Ising import Ising
# 使用 CIM 求解
test = Ising(J, h).solve(
    use_GPU=True,
    num_runs=20,
    num_timesteps_per_run=2000,
    cac_time_step=0.01,
    cac_r=-0.5,
    cac_alpha=0.8,
    cac_beta=0.3,
    cac_gamma=0.01,
    cac_delta=15,
    cac_mu=0.9,
    cac_rho=1.5,
    cac_tau=100,
    return_spin_trajectories_all_runs=True,
    hyperparameters_randomtune=True
)
# # 使用 CIM 求解
# test = Ising(J, h).solve(
#     use_GPU=True,
#     num_runs=50,  # 增加运行次数
#     num_timesteps_per_run=5000,  # 增加时间步
#     cac_time_step=0.02,  # 减小步长
#     cac_r=-0.5,  # 调整反馈强度
#     cac_alpha=0.8,
#     cac_beta=0.3,
#     cac_gamma=0.005,
#     cac_delta=15,
#     cac_mu=0.9,
#     cac_rho=1.5,
#     cac_tau=200,
#     return_spin_trajectories_all_runs=True,
#     hyperparameters_randomtune=False
# )

External Field Detected
Best combination of epsilon, lambda, and scaling constant: epsilon = 0.333; lambda = 0.1; scaling constant = 100.0
Target Ising Energy: -inf.
Best Ising Energy Found: -3.2490625.
Corresponding Spin Configuration: [-1.  1.  1.  1. -1. -1.  1.  1.  1.  1.].
Time Elapsed: 121.17846179008484.
Number of Runs Completed: 20.


In [37]:
from cim_optimizer.CIM_helper import brute_force

spins_ground, E_ground = brute_force(J)
print("The spin configuration in the ground state is {}".format(spins_ground))
print("The ground energy is {}".format(E_ground))

The spin configuration in the ground state is [-1 -1 -1 -1  1  1 -1 -1 -1 -1]
The ground energy is -2.375


In [39]:
# 调试：检查结果属性
print("test.result 属性:", [attr for attr in dir(test.result) if not attr.startswith('_')])

# 提取最低能量
best_energy = test.result.lowest_energy
print(f"最佳能量: {best_energy}")
# best_spins = spins_ground
best_spins = test.result.lowest_energy_spin_config
# best_spins = [1,  -1,  -1,  -1,  1,  -1,  -1,  -1, 1,  -1. -1,  1,  1,  1,  1,  1, -1,1]
print(f"最佳自旋配置: {best_spins}")

# 验证自旋长度
if len(best_spins) != N:
    print(f"错误：自旋长度 {len(best_spins)} 不等于预期 {N}")
    raise ValueError("自旋配置无效")

test.result 属性: ['energies', 'energy_evolution', 'lowest_energy', 'lowest_energy_spin_config', 'result_data', 'spin_config_all_runs', 'spin_trajectories', 'time']
最佳能量: -3.2490625
最佳自旋配置: [-1.  1.  1.  1. -1. -1.  1.  1.  1.  1.]


In [40]:
# 解码解
# 
x_sol = [(best_spins[i] + 1) / 2 for i in range(n * m)]
z_sol = [(best_spins[n * m + h] + 1) / 2 for h in range(l)]
tau_sol = [(best_spins[n * m + l + j * l + h] + 1) / 2 for j in range(m) for h in range(l)]
C_max = sum(2 ** h * z_sol[h] for h in range(l))
print(f"最大完工时间 (C_max): {C_max}")
print(f"分配方案 (x_ij): {x_sol}")

# 验证分配约束
x_matrix = np.array(x_sol).reshape(n, m)
print("分配矩阵:\n", x_matrix)
for i in range(n):
    if abs(sum(x_matrix[i]) - 1) > 1e-5:
        print(f"警告：作业 {i + 1} 分配不满足约束，sum(x_{i}j) = {sum(x_matrix[i])}")
    else:
        print(f"作业 {i + 1} 分配满足约束")

# 验证机器完工时间
for j in range(m):
    T_j = sum(x_matrix[i, j] * P[i, j] for i in range(n))
    print(f"机器 {j + 1} 完工时间: {T_j}")
    if T_j > C_max + 1e-5:
        print(f"警告：机器 {j + 1} 完工时间 {T_j} 超过 C_max {C_max}")

最大完工时间 (C_max): 0.0
分配方案 (x_ij): [0.0, 1.0, 1.0, 1.0]
分配矩阵:
 [[0. 1.]
 [1. 1.]]
作业 1 分配满足约束
警告：作业 2 分配不满足约束，sum(x_1j) = 2.0
机器 1 完工时间: 2.0
警告：机器 1 完工时间 2.0 超过 C_max 0.0
机器 2 完工时间: 3.0
警告：机器 2 完工时间 3.0 超过 C_max 0.0
